<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("IDN_1.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,freq,indexed_at,provider_code,series_code,series_name,series_num,REF_AREA
0,2010,2010-01-01,NA,annual,TABEL8_1,VIII.1 COMPOSITE CONSUMER PRICE INDEX OF 82 CI...,A,2022-04-04T15:16:01.661Z,BI,1.A,Foodstuff - Annual,Master,IDN
1,2011,2011-01-01,NA,annual,TABEL8_1,VIII.1 COMPOSITE CONSUMER PRICE INDEX OF 82 CI...,A,2022-04-04T15:16:01.661Z,BI,1.A,Foodstuff - Annual,Master,IDN
2,2012,2012-01-01,NA,annual,TABEL8_1,VIII.1 COMPOSITE CONSUMER PRICE INDEX OF 82 CI...,A,2022-04-04T15:16:01.661Z,BI,1.A,Foodstuff - Annual,Master,IDN
3,2013,2013-01-01,NA,annual,TABEL8_1,VIII.1 COMPOSITE CONSUMER PRICE INDEX OF 82 CI...,A,2022-04-04T15:16:01.661Z,BI,1.A,Foodstuff - Annual,Master,IDN
4,2014,2014-01-01,NA,annual,TABEL8_1,VIII.1 COMPOSITE CONSUMER PRICE INDEX OF 82 CI...,A,2022-04-04T15:16:01.661Z,BI,1.A,Foodstuff - Annual,Master,IDN


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [3]:
python_df = pd.read_pickle("IDN_1.pickle")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,freq,Frequency,REF_AREA
0,annual,BI,TABEL1_1,I.1. BROAD MONEY AND ITS AFFECTING FACTORS (Bi...,1.A,Broad Money (M2) - Annual,2010,2010-01-01,2471205.79,2471205.79,A,Annual,IDN
1,annual,BI,TABEL1_1,I.1. BROAD MONEY AND ITS AFFECTING FACTORS (Bi...,1.A,Broad Money (M2) - Annual,2011,2011-01-01,2877219.57,2877219.57,A,Annual,IDN
2,annual,BI,TABEL1_1,I.1. BROAD MONEY AND ITS AFFECTING FACTORS (Bi...,1.A,Broad Money (M2) - Annual,2012,2012-01-01,3307507.55,3307507.55,A,Annual,IDN
3,annual,BI,TABEL1_1,I.1. BROAD MONEY AND ITS AFFECTING FACTORS (Bi...,1.A,Broad Money (M2) - Annual,2013,2013-01-01,3730409.35,3730409.35,A,Annual,IDN
4,annual,BI,TABEL1_1,I.1. BROAD MONEY AND ITS AFFECTING FACTORS (Bi...,1.A,Broad Money (M2) - Annual,2014,2014-01-01,4173326.5,4173326.50,A,Annual,IDN


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [4]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [5]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce').astype('float64')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [18]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

False

<font size=5, font color='blue'> The provider reused the same series codes in multiple datasets. So now we need ['period_start_day','series_code','dataset_code'] to identify a unique row.

In [8]:
sum(python_df.duplicated(subset=cols))

4263

In [26]:
sum(stata_df.duplicated(subset=cols))

4263

In [28]:
a=stata_df[stata_df.duplicated(subset=cols,keep=False)]
a[(a['period_start_day']=='2010-01-01')&(a['series_code']=='1.A')].head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,freq,indexed_at,provider_code,series_code,series_name,series_num,REF_AREA
0,2010,2010-01-01,NaN,annual,TABEL8_1,VIII.1 COMPOSITE CONSUMER PRICE INDEX OF 82 CI...,A,2022-04-04T15:16:01.661Z,BI,1.A,Foodstuff - Annual,Master,IDN
531,2010,2010-01-01,4476999.76,annual,TABEL7_3,VII.3 GROSS DOMESTIC PRODUCT BY EXPENDITURES A...,A,2022-04-04T15:16:01.661Z,BI,1.A,Consumption Expenditures - Annual,Master,IDN
701,2010,2010-01-01,956119.70,annual,TABEL7_1,VII.1 GROSS DOMESTIC PRODUCT BY INDUSTRIAL ORI...,A,2022-04-04T15:16:01.661Z,BI,1.A,"AGRICULTURE, FORESTRY AND FISHERY - Annual",Master,IDN
1341,2010,2010-01-01,9142.51,annual,TABEL5_40,V.40. SELECTED FOREIGN EXCHANGE MIDDLE RATES A...,A,2022-04-04T15:16:01.661Z,BI,1.A,AUD - Annual,Master,IDN
1441,2010,2010-01-01,5144.29,annual,TABEL5_1,V.1. INDONESIA'S BALANCE OF PAYMENTS : SUMMARY...,A,2022-04-04T15:16:01.661Z,BI,1.A,I. Current Account - Annual,Master,IDN


In [29]:
sum(stata_df.duplicated(subset=cols+['dataset_code']))+sum(python_df.duplicated(subset=cols+['dataset_code'])) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code', 'dataset_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','dataset_code','value']] sliced from stata_df and python_df are exactly the same.

In [30]:
stata_sub = stata_df.sort_values(by=cols+['dataset_code'])[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols+['dataset_code'])[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True